In [34]:
import csv 
import json
import os
import sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from src.utils import parse_lipid, parse_p_value, ppm_tol

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats.multitest import fdrcorrection
from scipy.stats import linregress

In [2]:
colors = json.load(open(r'..\data\metadata\color_schemes.json'))
compound_superclasses = json.load(open('../data/metadata/compound_superclasses.json', 'r'))
ap = pd.read_excel(r'..\data\metadata\animal_phenotypes.xlsx', index_col=0)
fg = pd.read_csv(r'..\data\metadata\combined_metab_lipid_file_grouping.csv', index_col=0)

In [3]:
# Helper functions
def make_unique_name(name, rt, mz):
    return name + '_' + str(round(rt, 3)) + '_' + str(round(mz, 4))

def parse_tags(text):
    output = [
        ('mzCloudID' in text), 
        ('mzVaultID' in text), 
        ('ChemSpiderID' in text),
        ('Lipid' in text), 
        ('11.5' in text),
    ]
    return output

# def ppm_tol(mz1, mz2, tol=10):
#     return abs(mz1 - mz2) / mz1 * 1e6 < tol

# Lipid Data Processing


In [4]:
fr_raw = pd.read_csv(r'..\data\search\CD31\peak_finder\Final_Results_CD31.csv')
fr_raw = fr_raw.drop(fr_raw.filter(regex='Unnamed').columns, axis=1)  # filter out the "Unnamed: " column 
# log2 transform data
fr_raw.loc[:, fr_raw.filter(regex='Area \(max\)|\.raw').columns] = \
np.log2(fr_raw.loc[:, fr_raw.filter(regex='Area \(max\)|\.raw').columns])
# fr_raw

## Drop Alkenyl-DGs (very likely a mis-ID)

In [5]:
alkdgs = fr_raw.loc[(fr_raw['Identification'].notna()) & 
                    (fr_raw['Identification'].str.contains('Alkenyl'))].index
fr_raw.loc[alkdgs, 'Identification'] = np.nan
fr_raw.loc[alkdgs, 'Lipid Class'] = np.nan

In [6]:
## Drop 1091_9_FBG from fr_raw
# fr_raw = fr_raw.drop(fg.loc['1091_9_FBG', 'fr_name'], axis=1)

# Normalize to correct for run order bias

In [7]:
runorder = (fg
            .loc[fg['original']]
            .append(fg.loc['1091_9_FBG'])
            .sort_values('lipid_run_order')
            ['fr_name'])
file_medians = fr_raw[runorder].median(axis=0)
model = LinearRegression() \
    .fit(np.arange(0, file_medians.shape[0]).reshape(-1,1), file_medians)
slope = model.coef_[0]
intercept = model.intercept_
normalization_factor_arr = np.arange(0, len(runorder))*slope + intercept 
normalization_factor_arr = (normalization_factor_arr[0] / normalization_factor_arr)
fr_norm = fr_raw.copy()
fr_norm.loc[:, runorder] = fr_norm[runorder] * normalization_factor_arr

# Retain FAs (see `BJA-1.0.2`)

In [8]:
FAs = pd.read_csv(r'../data/interim/FAs_MS1_matched.csv', index_col=0)
fr_norm.loc[FAs.index, 'is_fa'] = True
fr_norm.loc[:, 'is_fa'].replace(np.nan, False, inplace=True)
fr_norm.loc[FAs.index, 'Identification'] = FAs.values
fr_norm.loc[FAs.index, 'Lipid Class'] = 'FA'
# fr_norm.loc[fr_norm['is_fa']]

# Many other tests for filtering lipids are in `BJA-1.1.1`, but the primary filters are:
1. High RSD compared to the QCs
1. low RT
2. small m/z
3. ratio of mean/blank < 3
3. Features found < 5

In [9]:
fr_norm['sample_mean'] = fr_norm[runorder].mean(axis=1)
fr_norm['blank_mean'] = fr_norm.filter(regex='Blank').mean(axis=1)
fr_norm['blank_max'] = fr_norm.filter(regex='Blank').max(axis=1)
fr_norm['sample_blank_diff'] = fr_norm['sample_mean'] - fr_norm['blank_mean']
fr_norm['sample_maxblank_diff'] = fr_norm['sample_mean'] - fr_norm['blank_max']

In [10]:
fr_final = fr_norm.copy()
fr_final = fr_final.loc[
  ~(
    (fr_norm['Retention Time (min)'] < 0.8) | 
    (fr_norm['Quant Ion'] < 350) | 
    (fr_norm['sample_maxblank_diff'] < 3) | 
    (fr_norm['Features Found'] < 5) 
   ) | (fr_norm['is_fa'] == True)
]
fr_final

,Retention Time (min),Quant Ion,Polarity,Area (max),Identification,Lipid Class,Features Found,20210729_AJ_Toh_RatBloodGlucose_ExtractionBlank.raw (F1),20210729_AJ_Toh_RatBloodGlucose_SolventBlank.raw (F2),20210729_AJ_Toh_RatBloodGlucose_T1060M_20210322_8wk_FBG.raw (F3),...,20210729_AJ_Toh_RatBloodGlucose_T1102M_20210413_10wk_RBG.raw (F64),20210729_AJ_Toh_RatBloodGlucose_Water_R1.raw (F65),20210729_AJ_Toh_RatBloodGlucose_Water_R2.raw (F66),20210729_AJ_Toh_RatBloodGlucose_Water_R3.raw (F67),is_fa,sample_mean,blank_mean,blank_max,sample_blank_diff,sample_maxblank_diff
67,0.818,622.29077,+,21.982179,NaN,NaN,19,12.941802,12.860211,20.182971,...,19.179026,13.015148,12.822537,12.804904,False,20.282531,12.901006,12.941802,7.381524,7.340729
75,0.871,453.16891,+,20.845767,NaN,NaN,19,15.573107,14.237157,20.067110,...,19.228641,14.003633,15.866233,15.239032,False,19.749192,14.905132,15.573107,4.844061,4.176086
76,0.873,645.11963,+,21.450707,NaN,NaN,7,14.087519,14.005928,20.045002,...,19.943100,14.185127,13.968254,13.950621,False,20.081812,14.046724,14.087519,6.035089,5.994293
77,0.876,631.34955,-,20.889744,NaN,NaN,10,12.929552,12.888105,18.343723,...,18.053283,13.326093,12.985003,12.902867,False,18.400726,12.908829,12.929552,5.491897,5.471174
84,0.933,376.25961,+,26.350005,NaN,NaN,58,12.595866,12.403226,23.817503,...,23.664533,12.755033,12.533462,12.467585,False,23.802752,12.499546,12.595866,11.303206,11.206886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1473,19.885,908.86542,+,23.050005,TG 18:0_18:0_18:0,TG,29,14.966498,15.312007,20.999142,...,20.648004,18.511598,16.680126,16.234939,False,20.379256,15.139252,15.312007,5.240004,5.067250
1479,20.144,948.89648,+,22.185651,NaN,NaN,7,13.641698,13.780943,19.774817,...,19.189126,14.638193,14.434339,13.820448,False,18.777856,13.711320,13.780943,5.066536,4.996913
1480,20.148,897.82532,+,21.562814,NaN,NaN,15,13.549287,13.688532,20.517490,...,17.314487,13.863481,14.341928,13.728038,False,19.001263,13.618910,13.688532,5.382353,5.312730
1482,20.175,593.58716,+,21.652712,NaN,NaN,19,13.716785,13.856031,20.603688,...,17.454831,13.960918,13.906388,13.895536,False,18.908803,13.786408,13.856031,5.122395,5.052772


## Add column `lc_type` for distinguishing that the ACs (which are lipids) came from the HILIC run

In [11]:
fr_final['lc_type'] = 'RP'
fr_final['Type'] = 'lipid'

# Rename columns

In [12]:
fr_final = fr_final.rename(
    {'Retention Time (min)': 'RT',
     'Quant Ion': 'm/z',
     'Polarity': 'polarity',
     'Identification': 'ID', 
     'Lipid Class': 'molec_class',
     **{old: new for old, new in zip(fg['fr_name'], fg.index)}
    }, axis=1
)
# fr_final

## Fill in ID, molec_class and superclass

In [13]:
fr_final['ID'] = fr_final['ID'].replace(np.nan, 'Unidentified')
fr_final['molec_class'] = fr_final['molec_class'].replace(np.nan, 'Unidentified')
fr_final['superclass'] = fr_final['molec_class'].map(compound_superclasses)
# Drop 3 ACs that get overlapping IDs from the HILIC data
fr_final = fr_final.loc[~(fr_final['molec_class'] == 'AC')]

# POLAR METABOLITES PROCESSING FROM COMPOUND DISCOVERER 

## Parse the Compounds Table and the Chemspider subtable together 

In [14]:
cd_path = '../data/CD33/Nilerat_metabs_20220516_compounds.csv'
cs_path = '../data/CD33/Nilerat_metabs_20220516_chemspider.csv'
cd_all = pd.read_csv(cd_path)
cs = []
with open(cs_path, encoding='utf-8-sig') as f:
    r = csv.reader(f)
    for line in r:
        cs.append(line) 
headers = cs[0]
cs_headers = cs[2]
ion_type_index = headers.index('Reference Ion')  # Get index of Reference Ion (e.g. [M+H]+1) in headers
cs_name_index = cs_headers.index('Name')         # Get index of chemspider name
df = []
for i, line in enumerate(cs[1:], start=1):
    cs_names = []
    if '[' in line[ion_type_index]:  # New Compound
        compound_line_index = i
        cs_line_index = compound_line_index + 2
#         print(cs[0])
        try:
            while cs[cs_line_index][1] == 'FALSE':
                cs_names.append(cs[cs_line_index][cs_name_index])
                cs_line_index += 1
        except IndexError:
            pass
        result = {header: entry for header, entry in zip(headers, cs[compound_line_index])}
        if not cs_names:
            cs_names = None
        else:
            cs_names = '\n'.join(cs_names)
        result['chemspider_matches'] = cs_names
        df.append(result)
    else:
        continue    
cd = pd.DataFrame(df)
# cd

## Parse columns

In [15]:
area_cols = cd.filter(regex='^Area: ').columns
cd[area_cols] = cd.loc[:, area_cols].astype('float')
cd.loc[:, area_cols] = np.log2(cd.loc[:, area_cols])
cd['lc_type'] = 'HILIC'
cd['polarity'] = cd['Reference Ion'].str[-2]
cd['Checked'] = cd['Checked'].map({'TRUE': True, 'FALSE': False})
cd['is_id'] = (cd['Name'] != '') & (cd['Checked'])
cd[['RT [min]', 'm/z']] = cd[['RT [min]', 'm/z']].astype('float')
cd['Type'] = 'metabolite'
cd['mzcloud'], cd['mzvault'], cd['chemspider'], cd['is_HILIC_lipid'], cd['11.5'] = \
    zip(*cd['Tags'].map(parse_tags))

## Assign ACs (see `BJA-1.0.2`)

In [16]:
ACs = pd.DataFrame(json.load(open(r'../data/interim/ACs_MS1_matched.json'))).set_index(0)
cd.loc[ACs.index, 'Name'] = ACs.values
cd.loc[ACs.index, 'confident_id'] = True
cd.loc[ACs.index, 'is_HILIC_lipid'] = False
cd.loc[ACs.index, 'Type'] = 'lipid'
cd.loc[ACs.index, 'molec_class'] = 'AC'
# cd.loc[ACs.index]

In [17]:
cd['confident_id'] = ((cd['mzvault'] | cd['mzcloud']) & \
                      (~cd['is_HILIC_lipid']) & \
                      (cd['Checked']) & \
                      (~cd['Name'].str.contains('drop this duplicate'))
                    ) | (cd['molec_class'] == 'AC')

## Parse Tracefinder ID names from YYZ's library

In [18]:
# Fix the metabolite names to be user-friendly and remove comments
fixed_metab_names = {
    "Acetyl-β-methylcholine": "Acetyl-beta-methylcholine",
    "Adenosine 5'-monophosphate": "Adenosine 5'-monophosphate",
    "Cytidine 5'-diphosphocholine": "Cytidine 5'-diphosphocholine",
    "DL-Carnitine": 'Carnitine',
    "DL-Glutamine": 'Glutamine',
    "DL-Leucine/Isoleucine": 'Leucine/Isoleucine',
    "DL-Lysine": 'Lysine',
    "DL-Proline": 'Proline',
    "Guanosine: all right MS2": 'Guanosine',
    "L-(+)-Arginine": 'Arginine',
    "L-(+)-Citrulline": 'Citrulline',
    "L(+)-Ornithine": 'Ornithine',
    "L-Alanine": 'Alanine',
    "L-Aspartic acid": 'Aspartic acid',
    "L-Glutamic acid": 'Glutamic acid',
    "L-Iditol to Six-carbon sugar alcohol": 'Iditol',
    "L-Phenylalanine": 'Phenylalanine',
    "L-Serine": 'Serine', 
    "L-Tyrosine": 'Tyrosine',
    "L-Valine": 'Valine',
    "O-Isovaleryl-L-carnitine: rt a little shift": 'AC 5:0',
    'O-Butyryl-L-carnitine': 'AC 4:0',
    'Acetyl-L-carnitine': 'AC 2:0',
    'Propionylcarnitine': 'AC 3:0',
    "?-Lactose": 'Lactose',
    "Xylitol to Arabitol": 'Xylitol',
    '?-Glutamylcysteine': 'Glutamylcysteine',
    'Uridinediphosphateglucose: standard RIAILs study': 'Uridinediphosphateglucose',
#     'Nicotinamide adenine dinucleotide (NAD+)': 'Nicotinamide adenine dinucleotide',
    'L-Pyroglutamic acid': 'Pyroglutamic acid',
    'Citric acid/Isocitric acid': 'Citric acid',
    'D-Hexose 1-phosphate': 'Hexose-1-phosphate',
    'L-Glutathione (reduced)': 'Glutathione',
    'L-2-Aminoadipic acid': '2-Aminoadipic acid',
    'N2-Acetyl-Lysine': 'N2-Acetyllysine',
    'L-Glutathione oxidized': 'Glutathione oxidized',
}
tf_rt_mz = pd.read_csv(r'..\data\interim\Tracefinder_RT_MZ_name.csv', encoding='utf-8', encoding_errors='ignore')
tf_rt_mz = tf_rt_mz.dropna(how='all', axis=1)
tf_rt_mz['Quan Mass'] = tf_rt_mz['Quan Mass'].str.strip(' mz').astype('float')
tf_rt_mz['Compound'] = tf_rt_mz['Compound'].map(fixed_metab_names).fillna(tf_rt_mz['Compound'])
tf_rt_mz = tf_rt_mz.set_index('Compound')
# display(tf_rt_mz)

## Find matches between TraceFinder targeted method and the IDs from CD 

In [19]:
cd['tracefinder_id'] = False
for i_cd, mrow in cd.iterrows():
    for compound_name, tfrow in tf_rt_mz.iterrows():
        if ppm_tol(mrow['m/z'], tfrow['Quan Mass'], tol=15) and abs(mrow['RT [min]'] - tfrow['RT']) < 1:
            # Threonine and Proline already confirmed
            if compound_name != 'Proline' and compound_name != 'Threonine':  
                cd.loc[i_cd, 'Name'] = compound_name
                cd.loc[i_cd, 'tracefinder_id'] = True
#             print(mrow['Name'], mrow['m/z'], mrow['RT [min]'], compound_name, tfrow['RT'])

## Remove any IDs that aren't from mzVault, or mzCloud, or Tracefinder, or ACs

In [20]:
not_confident = ~(cd['confident_id'] | cd['tracefinder_id'])  # | (cd['molec_class'] == 'AC')
cd['Name'] = cd['Name'].mask(cond=not_confident)
# cd.loc[cd['molec_class'] == 'AC']

## Remove HILIC lipids

In [21]:
cd = cd.loc[cd['is_HILIC_lipid'] == False]

## Rename columns

In [22]:
# Create a map from the filename with "Area: ... (F2)" to the shortened col name
metab_data_col_map = {}
for old_col in cd.filter(regex='^Area:').columns:
    interim_col = old_col.replace('Area: ', '').split('.raw')[0]
    try:
        shortened_col = fg.loc[fg['File Name_metab'] == interim_col].index[0]
    except: 
        continue
    if not isinstance(shortened_col, str):
        continue
    metab_data_col_map[old_col] = shortened_col

cd = cd.rename({'Name': 'ID', 'RT [min]': 'RT', **metab_data_col_map}, axis=1)

## Drop a few low confidence IDs and rename metabs

In [23]:
drop_metabs = ['Pentose sugar', 'Hexose sugar', 'Iditol', '1,5-Anhydro-D-glucitol']

rename_metabs = {
    'N6,N6,N6-Trimethyl-L-lysine': 'N6,N6,N6-Trimethyllysine',
    'N3,N4-Dimethyl-L-arginine': 'N3,N4-Dimethylarginine',
    'N-Acetyl-L-aspartic acid': 'N-Acetylaspartic acid',
    'Ethyl-beta-D-glucuronide': 'Ethylglucuronide',
    'Beta-D-Glucopyranuronic acid': 'Glucopyranuronic acid', 
    'Ascorbic acid 2-sulfate': 'Ascorbic acid sulfate',
    'Acetyl-β-methylcholine': 'Acetyl-beta-methylcholine',
    '4-Hydroxybutyric acid (GHB)': '4-Hydroxybutyric acid',
    '3-Indoxyl sulphate': 'Indoxyl sulfate',
}

cd['ID'] = cd['ID'].map(rename_metabs).fillna(cd['ID'])

for i, row in cd.iterrows():
    if row['ID'] in drop_metabs:
        cd.loc[i, ['ID', 'molec_class', 'superclass']] = 'Unidentified'
        cd.loc[i, ['confident_id', 'tracefinder_id']] = False

## Remove in-source frag or duplicate features

In [24]:
cd = cd.loc[~(cd['ID'] == 'Pyroglutamic acid')]
cd = cd.loc[~((cd['ID'] == 'Gluconic acid') & (cd['RT'] > 13.6) & (cd['RT'] < 13.7))]

## Assign Metabolite Molecular Class from list in `BJA-0.2.1`
## superclass will be same as molecular class for metabolites

In [25]:
cd['ID'] = cd['ID'].replace(np.nan, 'Unidentified')
cd['molec_class'] = cd['ID'].map(compound_superclasses)
cd.loc[cd['ID'].str.contains('AC'), 'molec_class'] = 'AC' 
cd['superclass'] = cd['molec_class'].map(compound_superclasses)
cd['superclass'].unique()

array(['Unidentified', 'Nitrogen heterocycle', 'Amino acid',
       'Organic acid', 'Carbohydrate', 'Amino acid derivative', 'Alcohol',
       'Fatty Acyl'], dtype=object)

# Combine lipid and metabolite dataframes

In [26]:
df = cd.append(fr_final)
df = df.reset_index()

# Keep select columns
cols_to_keep = ['ID', 'm/z', 'RT', 'molec_class', 'Type', 'polarity', 'lc_type', 'superclass']
cols_to_keep.extend(fg.loc[fg['original']].index)
df = df.loc[:, cols_to_keep]

## Drop `1091_9_FBG` due to outlier

In [27]:
## outlier is dropped because it was removed from "original" status in File Grouping
# df = df.drop('1091_9_FBG', axis=1)

# Make `unique_id` column and shortened `i` column

In [28]:
df['unique_id'] = df.apply(lambda x: make_unique_name(
    x['ID'],                                                    
    x['RT'],                             
    x['m/z']), axis=1)

df['i'] = None
for _type in ['lipid', 'metabolite']:
    counter = 0
    for i, row in df.loc[df['Type'] == _type].iterrows():
        df.loc[i, 'i'] = _type[0] + '_' + str(counter)
        counter += 1

### Drop unusual FAs

In [29]:
df = df.loc[~df['ID'].isin(['FA 27:3', 'FA 29:3', 'FA 31:3'])]

## Define non-fasted and fasted columns

In [30]:
rbg_cols = fg.loc[fg['original'] & (fg['bg_type'] == 'RBG')].index.to_list()
fbg_cols = fg.loc[fg['original'] & (fg['bg_type'] == 'FBG')].index.to_list()
# REMOVE OUTLIER SAMPLE
# fbg_cols.remove('1091_9_FBG')
data_cols = rbg_cols + fbg_cols

## Include log2 fold change of average of `nonfasted - fasted` for collaborators

In [31]:
gb = df.groupby(fg['bg_type'], axis=1).mean()
df['mean_sampling_log2fc'] = gb['RBG'] - gb['FBG']

# Do a bunch of linear models: 
1. Quant vs OGTT for each metabolite
    - Repeat for both 3-week-averaged and non-averaged data
    - `Quant ~ OGTT + sampling + OGTT : sampling`
    - Get from this model:
        1. 3 p-values, 
        2. $\beta$ regression slope
2. OGTT vs 1 metabolite Quant
    - Both 3 week averaged and non-averaged data
    - `OGTT ~ quant`
    - Get p-value and r2 and regression slope $\beta$ coef. from this model 
3. OGTT vs All metabolites Quant
    - Both 3-week-averaged and non-averaged data
    - #### This is done in machine learning modelling (Figure 4) 

In [81]:
noavg_lrdf = df.loc[:, data_cols].T.join(fg[['ogtt', 'bg_type', 'animal']])
avg_lrdf = noavg_lrdf.groupby(['bg_type', 'animal']).mean().reset_index()
avg_lrdf

,bg_type,animal,0,1,2,3,4,5,6,7,...,1403,1404,1405,1406,1407,1408,1409,1410,1411,ogtt
0,FBG,1060,19.384185,20.342467,23.747611,24.675900,21.175787,22.884704,22.365933,21.535287,...,18.478126,17.734779,19.078503,19.190682,20.803520,19.296442,20.252629,20.387191,19.783565,48742.5
1,FBG,1062,17.443496,20.618731,23.929733,24.351975,24.288569,22.828534,22.319930,21.846785,...,18.093069,17.238210,18.090474,18.484782,19.943932,18.167369,19.559432,19.492728,18.721313,43717.5
2,FBG,1074,17.921037,20.978108,22.920749,24.454637,25.451442,23.159768,22.845992,22.215595,...,19.020567,18.069863,18.910541,18.996897,19.947317,18.871539,19.316247,19.372435,18.091690,35250.0
3,FBG,1076,21.336339,21.123610,22.369026,23.990699,22.056913,22.776006,22.204800,21.654299,...,18.826895,18.375756,19.701053,19.391497,20.280191,18.454885,19.730483,19.532113,19.224106,60245.0
4,FBG,1082,21.755225,20.930137,23.927383,24.349442,23.970608,22.851289,22.441951,21.533222,...,16.659214,15.285513,16.651266,16.802309,19.599022,16.352400,19.173973,19.339970,18.866604,56462.5
5,FBG,1091,18.162726,20.677635,23.881969,24.800465,27.340973,23.099022,22.646017,21.893572,...,17.358290,16.444416,17.408786,17.790982,19.656729,17.436851,19.603454,19.543900,19.199281,19060.0
6,FBG,1092,16.786960,20.751905,22.978736,24.556693,24.935337,22.934484,22.484600,22.415765,...,17.627441,16.781597,16.505871,18.063114,19.708393,17.986876,19.587727,19.863669,19.259353,34022.5
7,FBG,1093,17.597364,20.731602,22.873846,24.526227,27.071283,23.125956,22.662139,22.294221,...,19.236080,18.458014,19.610172,19.821613,20.592500,20.179252,20.987275,21.016376,20.556835,25057.5
8,FBG,1101,17.178716,20.403285,22.995687,24.617242,23.796208,22.705584,22.569514,21.502851,...,18.702536,17.198510,17.818037,18.483779,20.125178,18.283789,19.149024,19.297452,18.666037,55007.5
9,FBG,1102,17.201230,20.646089,23.550555,24.786277,23.504298,22.945345,22.592770,21.783028,...,18.933531,18.004092,19.542458,19.146346,20.269367,18.891881,19.691228,19.829622,19.158372,46565.0


In [82]:
def fit_ogtt_vs_quant(feature, noavgdf, avgdf):

    results = {}
    
    for bg_type, bg_name, bg_cols in [('RBG', 'fed', rbg_cols), ('FBG', 'fasted', fbg_cols)]:
        # Non-averaged linear regression 
        x = noavgdf.loc[bg_cols, feature]
        y = noavgdf.loc[bg_cols, 'ogtt']
        lr = linregress(x=x, y=y)
        
        results[f'{bg_name}_noavg_ogtt_vs_quant_r2'] = lr.rvalue ** 2
        results[f'{bg_name}_noavg_ogtt_vs_quant_pval'] = lr.pvalue
        results[f'{bg_name}_noavg_ogtt_vs_quant_slope'] = lr.slope
        
        # Averaged linear regression
        x = avgdf.loc[avgdf['bg_type'] == bg_type, feature]
        y = avgdf.loc[avgdf['bg_type'] == bg_type, 'ogtt']
        lr = linregress(x=x, y=y)
        
        results[f'{bg_name}_avg_ogtt_vs_quant_r2'] = lr.rvalue ** 2
        results[f'{bg_name}_avg_ogtt_vs_quant_pval'] = lr.pvalue
        results[f'{bg_name}_avg_ogtt_vs_quant_slope'] = lr.slope
        
    return results

ogtt_vs_quant_results = {}

for feature in df.index:
    r = fit_ogtt_vs_quant(feature, noavgdf=noavg_lrdf, avgdf=avg_lrdf)
    ogtt_vs_quant_results[feature] = r
      
ogtt_vs_quant_results = pd.DataFrame(ogtt_vs_quant_results).T

In [83]:
ogtt_vs_quant_results

,fed_noavg_ogtt_vs_quant_r2,fed_noavg_ogtt_vs_quant_pval,fed_noavg_ogtt_vs_quant_slope,fed_avg_ogtt_vs_quant_r2,fed_avg_ogtt_vs_quant_pval,fed_avg_ogtt_vs_quant_slope,fasted_noavg_ogtt_vs_quant_r2,fasted_noavg_ogtt_vs_quant_pval,fasted_noavg_ogtt_vs_quant_slope,fasted_avg_ogtt_vs_quant_r2,fasted_avg_ogtt_vs_quant_pval,fasted_avg_ogtt_vs_quant_slope
0,0.298824,1.774542e-03,3228.924367,0.349335,0.071953,3774.713640,0.269437,0.003910,3454.345634,0.303333,0.098961,4280.508230
1,0.670758,3.159016e-08,18175.654494,0.766154,0.000908,20760.608149,0.001328,0.851133,1210.930360,0.004409,0.855393,3743.613014
2,0.020295,4.526725e-01,-2025.516333,0.107712,0.354534,-10750.290739,0.001194,0.858745,-571.632596,0.012573,0.757782,-2775.364235
3,0.024109,4.126169e-01,-5703.227063,0.047235,0.546405,-11174.005702,0.118526,0.067430,-13900.430465,0.259524,0.132570,-28982.082910
4,0.020436,4.510871e-01,-633.715151,0.063080,0.483959,-1956.097943,0.175843,0.023551,-1456.078039,0.730232,0.001637,-6008.656149
...,...,...,...,...,...,...,...,...,...,...,...,...
1407,0.468778,3.003979e-05,9110.674721,0.535780,0.016097,10412.854426,0.002331,0.803593,856.553001,0.023632,0.671558,5270.682609
1408,0.412178,1.311126e-04,7563.496362,0.528121,0.017275,9691.066271,0.047798,0.254531,-1911.906200,0.054510,0.516215,-3077.038399
1409,0.301763,1.666406e-03,7883.205466,0.431197,0.039157,11264.508082,0.081542,0.133203,-4450.907169,0.155705,0.259135,-9846.059889
1410,0.371961,3.461004e-04,8186.869675,0.467054,0.029355,10279.876717,0.072342,0.158286,-3836.536271,0.155717,0.259115,-9909.972765


In [32]:
def fit_anova_fixed_effects(feature, all_terms, debug=False):
    full_formula = f'{feature} ~ ' + ' + '.join(all_terms) 
    full_model = smf.ols(full_formula, data=data).fit()

    params = full_model.params.to_dict()
    params = {'model_param_' + key: value for key, value in params.items()}

    anova_df = sm.stats.anova_lm(full_model, typ=1)
    anova_df = anova_df.loc[~anova_df.index.isin(['Residual']), 'PR(>F)']
    anova_df = anova_df.to_dict()
    anova_df = {'pval_' + key: value for key, value in anova_df.items()}
    if debug: 
        return {**anova_df, **params}, full_model
    else:
        return {**anova_df, **params}

In [33]:
data = df.filter(regex='RBG|FBG', axis=1).set_index(df['i']).T

data = data.astype('float')
features = data.columns
data = data.join(fg[['bg_type', 'bg', 'week', 'gluc_tol', 'animal', 'ogtt', 'litter']])

# ### ENCODING WEEK AND LITTER AS A CATEGORICAL STRING INSTEAD OF NUMERIC ###
# data[['week', 'litter']] = data[['week', 'litter']].astype('str')  

# weight_insulin = fg['animal'].apply(lambda x: ap.loc[x, ['Insulin (AUC)', 'Weightprefastweek12']])
# data = data.join(weight_insulin)
data.rename({'bg_type': 'sampling', 
             'Insulin (AUC)': 'insulin', 
             'Weightprefastweek12': 'weight'}, 
            inplace=True, axis=1)
data

,m_0,m_1,m_2,m_3,m_4,m_5,m_6,m_7,m_8,m_9,...,l_914,l_915,l_916,sampling,bg,week,gluc_tol,animal,ogtt,litter
1091_8_FBG,18.724636,20.779580,24.180859,24.887680,27.518513,23.065294,22.586018,21.670779,22.690034,22.292465,...,19.073429,19.091184,18.620317,FBG,53.333333,8.0,normal,1091,19060.0,2.0
1091_10_FBG,17.600816,20.575689,23.583080,24.713250,27.163433,23.132749,22.706016,22.116365,22.690580,21.771079,...,20.133479,19.996615,19.778245,FBG,48.500000,10.0,normal,1091,19060.0,2.0
1093_8_FBG,16.451618,20.902141,23.548337,24.647222,27.405718,23.138487,22.633815,22.139245,22.665739,22.337209,...,21.315704,21.404572,21.063577,FBG,47.000000,8.0,normal,1093,25057.5,2.0
1093_9_FBG,18.286166,20.877460,22.368788,24.526725,27.051667,23.178672,22.559027,22.469655,22.629048,22.274394,...,21.529536,21.430516,20.968101,FBG,64.000000,9.0,normal,1093,25057.5,2.0
1093_10_FBG,18.054307,20.415204,22.704412,24.404735,26.756463,23.060708,22.793575,22.273762,22.923587,21.875123,...,20.116586,20.214040,19.638826,FBG,56.500000,10.0,normal,1093,25057.5,2.0
1060_8_FBG,17.594730,20.198771,23.895235,24.714910,17.148994,22.653452,22.084504,21.397206,22.065355,21.679060,...,20.517490,20.603688,19.836798,FBG,41.000000,8.0,impaired,1060,48742.5,0.0
1060_9_FBG,21.809716,20.524661,24.263211,24.520405,19.169210,23.207043,22.450039,21.424020,22.534742,21.849560,...,20.671826,20.788152,20.234283,FBG,48.000000,9.0,impaired,1060,48742.5,0.0
1060_10_FBG,18.748109,20.303969,23.084387,24.792384,27.209158,22.793616,22.563256,21.784634,22.664376,21.631447,...,19.568571,19.769735,19.279616,FBG,52.500000,10.0,impaired,1060,48742.5,0.0
1062_8_FBG,18.098242,21.019048,24.873347,24.787384,21.774699,23.221875,22.352220,21.964129,22.346725,22.098343,...,19.348489,19.240306,18.585339,FBG,52.333333,8.0,impaired,1062,43717.5,0.0
1062_9_FBG,17.286972,20.641492,24.131393,24.408454,23.189693,22.831914,22.452734,21.841118,22.588667,21.916517,...,19.373546,19.419154,18.670027,FBG,48.000000,9.0,impaired,1062,43717.5,0.0


In [34]:
ogtt_sampling_terms = ['ogtt', 'sampling',  'sampling:ogtt']

ogtt_fer = []  # fer = Fixed Effects Result
for feature in features:
    ogtt_fer.append(fit_anova_fixed_effects(feature=feature, all_terms=ogtt_sampling_terms))
ogtt_fer = pd.DataFrame(ogtt_fer, index=features)


### ADDITIONAL CODE TO DO LIKELIHOOD RATIO TEST FOR (INSULIN OGTT):SAMPLING TERMS 

# itt_sampling_terms =  ['insulin', 'sampling', 'sampling:insulin']
# itt_fer = [] 
# for feature in features:
#     itt_fer.append(fit_anova_fixed_effects(feature=feature, all_terms=itt_sampling_terms))
# itt_fer = pd.DataFrame(itt_fer, index=features)

## Perform 2 separate linear regression fits on only Fed data, and only Fasted data for each feature

In [35]:
# One example of a linear regression fit 
fit = smf.ols('l_762 ~ ogtt', data=data, subset=rbg_cols).fit()
fit.pvalues['ogtt']

def get_linear_regression(feature):
    d = {}
    for cols, bg_type in [(rbg_cols, 'fed'), (fbg_cols, 'fasted')]:
        fit = smf.ols(f'{feature} ~ ogtt', data=data, subset=cols).fit()
        d['pval_' + bg_type] = fit.pvalues['ogtt']
        d['coef_' + bg_type] = fit.params['ogtt']
        d['intercept_' + bg_type] = fit.params['Intercept']
        
    # display(fit.summary())
    return d

feature = 'm_1'
get_linear_regression(feature)

{'pval_fed': 3.1590160514400507e-08,
 'coef_fed': 3.69042256293342e-05,
 'intercept_fed': 20.089207679864238,
 'pval_fasted': 0.8511333563584583,
 'coef_fasted': 1.0966543412315325e-06,
 'intercept_fasted': 20.674434437962763}

In [36]:
fits = []
for feature in features:
    fits.append(get_linear_regression(feature))
fits = pd.DataFrame(fits, index=features)
fits

,pval_fed,coef_fed,intercept_fed,pval_fasted,coef_fasted,intercept_fasted
i,,,,,,
m_0,1.774542e-03,0.000093,14.839852,0.003910,0.000078,15.116552
m_1,3.159016e-08,0.000037,20.089208,0.851133,0.000001,20.674434
m_2,4.526725e-01,-0.000010,23.382609,0.858745,-0.000002,23.388362
m_3,4.126169e-01,-0.000004,23.982960,0.067430,-0.000009,24.869487
m_4,4.510871e-01,-0.000032,26.522464,0.023551,-0.000121,29.475577
...,...,...,...,...,...,...
l_912,3.003979e-05,0.000051,18.452536,0.803593,0.000003,19.990039
l_913,1.311126e-04,0.000054,16.772587,0.254531,-0.000025,19.505542
l_914,1.666406e-03,0.000038,16.665978,0.133203,-0.000018,20.500428


In [37]:
# FDR correction
# Adds FDR-corrected column to ogtt_fer and fits DataFrames 
for fer in [ogtt_fer, fits]:  # itt_fer
    for i, pval_col in enumerate(fer.filter(regex='pval')):
        fdr_col_name = 'qval_' + pval_col.split('pval_')[1]
        fer.insert(i, fdr_col_name, fdrcorrection(fer[pval_col])[1])

# Combine DFs

In [38]:
data = df.join(ogtt_fer, on=df['i']).set_index('unique_id')
data = data.join(fits, on=data['i'])

# Make useful columns for later processing 

In [39]:
data['signif_interaction'] = data['qval_sampling:ogtt'] < 0.05
data['signif_sampling'] = data['qval_sampling'] < 0.05
gb_means = (data
            .loc[:, data_cols]
            .groupby(fg['bg_type'], axis=1)
            .mean()
           )

data['fasted_mean'] = gb_means['FBG']
data['fed_mean'] = gb_means['RBG']
data['Log2 Fold Change'] = data['fed_mean'] - data['fasted_mean']

data['Fed - Fasted slope'] = data['coef_fed'] - data['coef_fasted']
data['signif_sampling'] = data['qval_sampling'] < 0.05
data['signif_interact'] = data['qval_sampling:ogtt'] < 0.05
data['log_qval_sampling'] = -np.log10(data['qval_sampling'])
data['log_qval_ogtt'] = -np.log10(data['qval_ogtt'])
data['log_qval_sampling:ogtt'] = -np.log10(data['qval_sampling:ogtt'])
data['is_id'] = data['superclass'] != 'Unidentified'

l_ids = data.loc[(data['Type'] == 'lipid') & (data['ID'] != 'unknown')].index
data.loc[l_ids, 'lipid_class']     = data.loc[l_ids, 'ID'].apply(lambda x: parse_lipid(x)[0])
data.loc[l_ids, 'extra_lipid_label']     = data.loc[l_ids, 'ID'].apply(lambda x: parse_lipid(x)[1])
data.loc[l_ids, 'fa_carbons']      = data.loc[l_ids, 'ID'].apply(lambda x: parse_lipid(x)[2])
data.loc[l_ids, 'fa_unsat']        = data.loc[l_ids, 'ID'].apply(lambda x: parse_lipid(x)[3])
data.loc[l_ids, 'fa_carbon:unsat'] = data.loc[l_ids, 'ID'].apply(lambda x: parse_lipid(x)[4])
data['pval_asterisks']  = data['qval_sampling:ogtt'].apply(lambda x: parse_p_value(x))

# Combine and write to files

In [40]:
data.to_csv(r'..\data\processed\combined_metabolites_data_with_model_params.csv')
display(data)
data.drop(data.filter(regex='model_param|coef_|intercept_').columns, axis=1, inplace=True)
display(data)
data.to_csv(r'..\data\processed\combined_metabolites_data.csv')
data.to_excel(r'..\data\processed\combined_metabolites_data.xlsx')

,ID,m/z,RT,molec_class,Type,polarity,lc_type,superclass,1091_8_FBG,1091_10_FBG,...,log_qval_sampling,log_qval_ogtt,log_qval_sampling:ogtt,is_id,lipid_class,extra_lipid_label,fa_carbons,fa_unsat,fa_carbon:unsat,pval_asterisks
unique_id,,,,,,,,,,,,,,,,,,,,,
Unidentified_2.353_141.0659,Unidentified,141.06591,2.353,Unidentified,metabolite,-,HILIC,Unidentified,18.724636,17.600816,...,0.209851,3.911627,0.092460,False,NaN,NaN,NaN,NaN,NaN,ns
Unidentified_11.539_180.0585,Unidentified,180.05850,11.539,Unidentified,metabolite,-,HILIC,Unidentified,20.779580,20.575689,...,11.987252,4.759830,3.239845,False,NaN,NaN,NaN,NaN,NaN,***
Urocanic acid_11.365_139.0503,Urocanic acid,139.05028,11.365,Nitrogen heterocycle,metabolite,+,HILIC,Nitrogen heterocycle,24.180859,23.583080,...,0.748819,0.233844,0.104163,True,NaN,NaN,NaN,NaN,NaN,ns
Uridine_7.513_243.0615,Uridine,243.06152,7.513,Nitrogen heterocycle,metabolite,-,HILIC,Nitrogen heterocycle,24.887680,24.713250,...,9.482876,0.889251,0.159312,True,NaN,NaN,NaN,NaN,NaN,ns
Uric acid_12.732_167.0201,Uric acid,167.02007,12.732,Nitrogen heterocycle,metabolite,-,HILIC,Nitrogen heterocycle,27.518513,27.163433,...,0.459055,1.250723,0.480416,True,NaN,NaN,NaN,NaN,NaN,ns
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TG 18:0_18:0_18:0_19.885_908.8654,TG 18:0_18:0_18:0,908.86542,19.885,TG,lipid,+,RP,Glycerolipid,19.413597,19.899861,...,1.893991,2.856877,1.814371,True,TG,,54.0,0.0,54:0,*
Unidentified_20.144_948.8965,Unidentified,948.89648,20.144,Unidentified,lipid,+,RP,Unidentified,16.606829,18.266873,...,1.258580,0.595997,1.828730,False,NaN,NaN,NaN,NaN,NaN,*
Unidentified_20.148_897.8253,Unidentified,897.82532,20.148,Unidentified,lipid,+,RP,Unidentified,19.073429,20.133479,...,7.730918,0.617574,2.049913,False,NaN,NaN,NaN,NaN,NaN,**


,ID,m/z,RT,molec_class,Type,polarity,lc_type,superclass,1091_8_FBG,1091_10_FBG,...,log_qval_sampling,log_qval_ogtt,log_qval_sampling:ogtt,is_id,lipid_class,extra_lipid_label,fa_carbons,fa_unsat,fa_carbon:unsat,pval_asterisks
unique_id,,,,,,,,,,,,,,,,,,,,,
Unidentified_2.353_141.0659,Unidentified,141.06591,2.353,Unidentified,metabolite,-,HILIC,Unidentified,18.724636,17.600816,...,0.209851,3.911627,0.092460,False,NaN,NaN,NaN,NaN,NaN,ns
Unidentified_11.539_180.0585,Unidentified,180.05850,11.539,Unidentified,metabolite,-,HILIC,Unidentified,20.779580,20.575689,...,11.987252,4.759830,3.239845,False,NaN,NaN,NaN,NaN,NaN,***
Urocanic acid_11.365_139.0503,Urocanic acid,139.05028,11.365,Nitrogen heterocycle,metabolite,+,HILIC,Nitrogen heterocycle,24.180859,23.583080,...,0.748819,0.233844,0.104163,True,NaN,NaN,NaN,NaN,NaN,ns
Uridine_7.513_243.0615,Uridine,243.06152,7.513,Nitrogen heterocycle,metabolite,-,HILIC,Nitrogen heterocycle,24.887680,24.713250,...,9.482876,0.889251,0.159312,True,NaN,NaN,NaN,NaN,NaN,ns
Uric acid_12.732_167.0201,Uric acid,167.02007,12.732,Nitrogen heterocycle,metabolite,-,HILIC,Nitrogen heterocycle,27.518513,27.163433,...,0.459055,1.250723,0.480416,True,NaN,NaN,NaN,NaN,NaN,ns
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TG 18:0_18:0_18:0_19.885_908.8654,TG 18:0_18:0_18:0,908.86542,19.885,TG,lipid,+,RP,Glycerolipid,19.413597,19.899861,...,1.893991,2.856877,1.814371,True,TG,,54.0,0.0,54:0,*
Unidentified_20.144_948.8965,Unidentified,948.89648,20.144,Unidentified,lipid,+,RP,Unidentified,16.606829,18.266873,...,1.258580,0.595997,1.828730,False,NaN,NaN,NaN,NaN,NaN,*
Unidentified_20.148_897.8253,Unidentified,897.82532,20.148,Unidentified,lipid,+,RP,Unidentified,19.073429,20.133479,...,7.730918,0.617574,2.049913,False,NaN,NaN,NaN,NaN,NaN,**
